# Powell vs CCD-Accel: Experiment Protocol
**Requirements**: Powell method, 1D line search (golden-section in fixed [a,b] and backtracking/Armijo),
runs on Ackley/Branin (Powell) and Rosenbrock/Ackley (Powell vs CCD-Accel),
console table, trajectory & convergence plots, identical line-search across methods.

In [1]:
# Cell 1: Imports & Utilities (english annotations as requested)
import numpy as np
import math, time, os, zipfile
import matplotlib.pyplot as plt
from dataclasses import dataclass, field
from typing import Callable, Tuple, List
import pandas as pd

# Global matplotlib settings (no seaborn, no custom styles/colors)
plt.rcParams.update({"figure.dpi": 120})

def norm(x: np.ndarray) -> float:
    return float(np.linalg.norm(x))

def savefig(path: str):
    os.makedirs(os.path.dirname(path), exist_ok=True)
    plt.savefig(path, bbox_inches='tight')
    plt.close()

In [2]:
# Cell 2: Function evaluation wrapper (counts & history)
@dataclass
class FWrap:
    f: Callable[[np.ndarray], float]
    name: str
    evals: int = 0
    history_x: List[np.ndarray] = field(default_factory=list)
    history_f: List[float] = field(default_factory=list)
    def __call__(self, x: np.ndarray) -> float:
        fx = float(self.f(x))
        self.evals += 1
        self.history_x.append(np.array(x, dtype=float))
        self.history_f.append(fx)
        return fx
    def reset(self):
        self.evals = 0
        self.history_x.clear()
        self.history_f.clear()

In [3]:
# Cell 3: Test functions (Ackley, Branin, Rosenbrock) and known minima
def ackley(xy: np.ndarray) -> float:
    x, y = xy
    term1 = -20.0 * math.exp(-0.2 * math.sqrt(0.5*(x*x + y*y)))
    term2 = -math.exp(0.5*(math.cos(2*math.pi*x)+math.cos(2*math.pi*y)))
    return term1 + term2 + math.e + 20.0

def branin(xy: np.ndarray) -> float:
    x, y = xy
    a = 1.0
    b = 5.1/(4*math.pi**2)
    c = 5.0/math.pi
    r = 6.0
    s = 10.0
    t = 1.0/(8*math.pi)
    return a*(y - b*x*x + c*x - r)**2 + s*(1 - t)*math.cos(x) + s

def rosenbrock(xy: np.ndarray) -> float:
    x, y = xy
    return 100.0*(y - x*x)**2 + (1 - x)**2

KNOWN_MINIMA = {
    'ackley': [(0.0, 0.0)],
    'branin': [(math.pi, 2.275)],  # one of the three global minima
    'rosenbrock': [(1.0, 1.0)]
}

In [4]:
# Cell 4: Line searches (Golden-section with fixed [a,b], Backtracking/Armijo)
def line_search_golden(f: Callable[[np.ndarray], float], x: np.ndarray, d: np.ndarray,
                       a: float = -50.0, b: float = 50.0, tol: float = 1e-6):
    """Golden-section along {x + alpha d}. Kept in the user's required [a,b] fixed form."""
    golden_ratio = (math.sqrt(5) - 1) / 2  # ~0.618
    def phi(alpha: float) -> float:
        return f(x + alpha * d)
    alpha1 = a + (1 - golden_ratio) * (b - a)
    alpha2 = a + golden_ratio * (b - a)
    f1 = phi(alpha1)
    f2 = phi(alpha2)
    while abs(b - a) > tol:
        if f1 < f2:
            b = alpha2
            alpha2 = alpha1
            f2 = f1
            alpha1 = a + (1 - golden_ratio) * (b - a)
            f1 = phi(alpha1)
        else:
            a = alpha1
            alpha1 = alpha2
            f1 = f2
            alpha2 = a + golden_ratio * (b - a)
            f2 = phi(alpha2)
    alpha_star = 0.5*(a + b)
    x_new = x + alpha_star * d
    return x_new, f(x_new), alpha_star

def line_search_backtracking(f: Callable[[np.ndarray], float], x: np.ndarray, d: np.ndarray,
                             alpha0: float = 1.0, rho: float = 0.5, c: float = 1e-4, max_backtracks: int = 60):
    """Armijo backtracking without gradients (use secant slope proxy)."""
    fx = f(x)
    eps = 1e-6 / max(1.0, norm(d))  # finite-difference step
    slope_proxy = (f(x + eps * d) - fx) / eps
    alpha = alpha0
    for _ in range(max_backtracks):
        x_new = x + alpha * d
        if f(x_new) <= fx + c * alpha * slope_proxy:
            return x_new, f(x_new), alpha
        alpha *= rho
    x_new = x + alpha * d
    return x_new, f(x_new), alpha

In [5]:
# Cell 5: Optimizers (Powell; CCD with one acceleration step)
@dataclass
class OptResult:
    method: str
    x0: np.ndarray
    x: np.ndarray
    f: float
    nit_outer: int
    fevals: int
    time_sec: float
    history_x: List[np.ndarray]
    history_f: List[float]

def powell(fwrap: FWrap, x0: np.ndarray, tol: float = 1e-6, max_outer: int = 200, linesearch: str = 'golden') -> OptResult:
    n = len(x0)
    U = [np.eye(n)[i].copy() for i in range(n)]  # initial basis
    x = x0.astype(float).copy()
    t0 = time.time()
    hist_x = [x.copy()]
    hist_f = [fwrap(x)]
    ls = line_search_golden if linesearch=='golden' else line_search_backtracking
    nit = 0
    while nit < max_outer:
        x_prev = x.copy()
        for i in range(n):
            x, fx, _ = ls(fwrap, x, U[i])
            hist_x.append(x.copy()); hist_f.append(fx)
        # shift directions and append displacement
        for i in range(n-1):
            U[i] = U[i+1].copy()
        U[-1] = (x - x_prev)
        # extra search along displacement
        x, fx, _ = ls(fwrap, x, U[-1])
        hist_x.append(x.copy()); hist_f.append(fx)
        if norm(x - x_prev) <= tol:
            break
        nit += 1
    t1 = time.time() - t0
    return OptResult(method=f'Powell({linesearch})', x0=x0, x=x, f=fwrap(x),
                     nit_outer=nit, fevals=fwrap.evals, time_sec=t1,
                     history_x=hist_x, history_f=hist_f)

def ccd_accel(fwrap: FWrap, x0: np.ndarray, tol: float = 1e-6, max_outer: int = 200, linesearch: str = 'golden') -> OptResult:
    n = len(x0)
    x = x0.astype(float).copy()
    t0 = time.time()
    hist_x = [x.copy()]
    hist_f = [fwrap(x)]
    ls = line_search_golden if linesearch=='golden' else line_search_backtracking
    nit = 0
    while nit < max_outer:
        x_prev = x.copy()
        for i in range(n):
            ei = np.zeros(n); ei[i] = 1.0
            x, fx, _ = ls(fwrap, x, ei)
            hist_x.append(x.copy()); hist_f.append(fx)
        # acceleration along displacement
        d = x - x_prev
        x, fx, _ = ls(fwrap, x, d)
        hist_x.append(x.copy()); hist_f.append(fx)
        if norm(x - x_prev) <= tol:
            break
        nit += 1
    t1 = time.time() - t0
    return OptResult(method=f'CCD-Accel({linesearch})', x0=x0, x=x, f=fwrap(x),
                     nit_outer=nit, fevals=fwrap.evals, time_sec=t1,
                     history_x=hist_x, history_f=hist_f)

In [6]:
# Cell 6: Plotting
def make_grid_bounds(start: np.ndarray, minima: List[tuple], pad: float = 2.0):
    xs = [start[0]] + [m[0] for m in minima]
    ys = [start[1]] + [m[1] for m in minima]
    return min(xs)-pad, max(xs)+pad, min(ys)-pad, max(ys)+pad

def plot_trajectory(fun: Callable[[np.ndarray], float], res: OptResult, fname: str,
                    minima: List[tuple], explicit_bounds=None, grid_N: int = 300):
    xs = np.array([p[0] for p in res.history_x])
    ys = np.array([p[1] for p in res.history_x])
    xmin, xmax, ymin, ymax = make_grid_bounds(res.x0, minima) if explicit_bounds is None else explicit_bounds
    X = np.linspace(xmin, xmax, grid_N)
    Y = np.linspace(ymin, ymax, grid_N)
    XX, YY = np.meshgrid(X, Y)
    ZZ = np.zeros_like(XX)
    for i in range(grid_N):
        for j in range(grid_N):
            ZZ[i, j] = fun(np.array([XX[i, j], YY[i, j]]))
    plt.figure()
    plt.contour(XX, YY, ZZ, levels=30)
    plt.plot(xs, ys, marker='o', linewidth=1)
    plt.scatter([res.x0[0]], [res.x0[1]], marker='s', s=40, label='start')
    plt.scatter([res.x[0]], [res.x[1]], marker='*', s=80, label='final')
    for (mx, my) in minima:
        plt.scatter([mx], [my], marker='x', s=50, label='global min')
    plt.legend()
    plt.title(f'Trajectory: {res.method}')
    plt.xlabel('x'); plt.ylabel('y')
    savefig(fname)

def plot_convergence(res: OptResult, fname: str):
    plt.figure()
    plt.plot(res.history_f, marker='o', linewidth=1)
    plt.xlabel('iteration (line-search steps)')
    plt.ylabel('f(x)')
    plt.title(f'Convergence: {res.method}')
    savefig(fname)

In [7]:
# Cell 7: Experiment A — Powell on Ackley & Branin (fixed starts from PDF)
tol = 1e-6
max_outer = 200
suites_A = [('ackley', ackley, np.array([-3.0, -3.0])),
            ('branin', branin, np.array([2.0,  2.0]))]

rows_A = []
for ls_name in ['golden', 'backtracking']:
    for name, ffun, x0 in suites_A:
        fw = FWrap(ffun, name)
        res = powell(fw, x0, tol=tol, max_outer=max_outer, linesearch=ls_name)
        traj_png = f"/mnt/data/plot {name} trajectory_{ls_name}.png"
        conv_png = f"/mnt/data/plot {name} convergence_{ls_name}.png"
        plot_trajectory(ffun, res, traj_png, KNOWN_MINIMA[name])
        plot_convergence(res, conv_png)
        rows_A.append({
            'function': name, 'start': tuple(x0.tolist()),
            'method': 'Powell', 'line_search': ls_name,
            'x found': tuple(res.x.tolist()), 'f(x found)': res.f,
            'iterations': res.nit_outer, 'f evals': res.fevals, 'time (s)': res.time_sec,
            'trajectory_png': traj_png, 'convergence_png': conv_png
        })
df_powell_AB = pd.DataFrame(rows_A)
df_powell_AB

C:\Users\LI\AppData\Local\Temp\ipykernel_13544\3094227993.py:4: RuntimeWarning: overflow encountered in scalar add
  term1 = -20.0 * math.exp(-0.2 * math.sqrt(0.5*(x*x + y*y)))
C:\Users\LI\AppData\Local\Temp\ipykernel_13544\3094227993.py:4: RuntimeWarning: overflow encountered in scalar multiply
  term1 = -20.0 * math.exp(-0.2 * math.sqrt(0.5*(x*x + y*y)))


,function,start,method,line_search,x found,f(x found),iterations,f evals,time (s),trajectory_png,convergence_png
0,ackley,"(-3.0, -3.0)",Powell,golden,"(1.3043479338803706e+207, 1.147520204080763e+207)",20.758108,200,25202,0.581024,/mnt/data/plot ackley trajectory_golden.png,/mnt/data/plot ackley convergence_golden.png
1,branin,"(2.0, 2.0)",Powell,golden,"(9.424777971305316, 2.4750000089724855)",0.397887,3,506,0.022445,/mnt/data/plot branin trajectory_golden.png,/mnt/data/plot branin convergence_golden.png
2,ackley,"(-3.0, -3.0)",Powell,backtracking,"(-0.96875, 0.03125000000000196)",2.613604,2,203,0.012188,/mnt/data/plot ackley trajectory_backtracking.png,/mnt/data/plot ackley convergence_backtracking...
3,branin,"(2.0, 2.0)",Powell,backtracking,"(3.125, 2.5625000000000018)",0.474573,1,134,0.000000,/mnt/data/plot branin trajectory_backtracking.png,/mnt/data/plot branin convergence_backtracking...


In [8]:
# Cell 8: Experiment B — Powell vs CCD-Accel on Rosenbrock & Ackley (same starts, same line-search)
tol = 1e-6
max_outer = 300
suites_B = [('rosenbrock', rosenbrock, np.array([-1.5, 2.0])),
            ('ackley', ackley, np.array([4.0, 1.0]))]

rows_B = []
for ls_name in ['golden', 'backtracking']:
    for name, ffun, x0 in suites_B:
        # Powell
        fw1 = FWrap(ffun, name)
        res1 = powell(fw1, x0, tol=tol, max_outer=max_outer, linesearch=ls_name)
        traj_png1 = f"/mnt/data/plot {name} trajectory_powell_{ls_name}.png"
        conv_png1 = f"/mnt/data/plot {name} convergence_powell_{ls_name}.png"
        plot_trajectory(ffun, res1, traj_png1, KNOWN_MINIMA[name])
        plot_convergence(res1, conv_png1)
        rows_B.append({
            'function': name, 'start': tuple(x0.tolist()), 'method': 'Powell', 'line_search': ls_name,
            'x found': tuple(res1.x.tolist()), 'f(x found)': res1.f,
            'iterations': res1.nit_outer, 'f evals': res1.fevals, 'time (s)': res1.time_sec,
            'trajectory_png': traj_png1, 'convergence_png': conv_png1
        })
        # CCD-Accel
        fw2 = FWrap(ffun, name)
        res2 = ccd_accel(fw2, x0, tol=tol, max_outer=max_outer, linesearch=ls_name)
        traj_png2 = f"/mnt/data/plot {name} trajectory_ccd_{ls_name}.png"
        conv_png2 = f"/mnt/data/plot {name} convergence_ccd_{ls_name}.png"
        plot_trajectory(ffun, res2, traj_png2, KNOWN_MINIMA[name])
        plot_convergence(res2, conv_png2)
        rows_B.append({
            'function': name, 'start': tuple(x0.tolist()), 'method': 'CCD-Accel', 'line_search': ls_name,
            'x found': tuple(res2.x.tolist()), 'f(x found)': res2.f,
            'iterations': res2.nit_outer, 'f evals': res2.fevals, 'time (s)': res2.time_sec,
            'trajectory_png': traj_png2, 'convergence_png': conv_png2
        })
df_cmp = pd.DataFrame(rows_B)
df_cmp

,function,start,method,line_search,x found,f(x found),iterations,f evals,time (s),trajectory_png,convergence_png
0,rosenbrock,"(-1.5, 2.0)",Powell,golden,"(0.9812176880893414, 0.9650433999572972)",8.613898e-04,8,1136,0.015967,/mnt/data/plot rosenbrock trajectory_powell_go...,/mnt/data/plot rosenbrock convergence_powell_g...
1,rosenbrock,"(-1.5, 2.0)",CCD-Accel,golden,"(0.8734716540187354, 0.762228461536349)",1.606188e-02,300,37802,0.531466,/mnt/data/plot rosenbrock trajectory_ccd_golde...,/mnt/data/plot rosenbrock convergence_ccd_gold...
2,ackley,"(4.0, 1.0)",Powell,golden,"(-1.033061574709855e-07, 5.763570736855332e-08)",3.345931e-07,1,254,0.004194,/mnt/data/plot ackley trajectory_powell_golden...,/mnt/data/plot ackley convergence_powell_golde...
3,ackley,"(4.0, 1.0)",CCD-Accel,golden,"(-1.0330673178492949e-07, 5.7635143418631584e-08)",3.345937e-07,1,254,0.003433,/mnt/data/plot ackley trajectory_ccd_golden.png,/mnt/data/plot ackley convergence_ccd_golden.png
4,rosenbrock,"(-1.5, 2.0)",Powell,backtracking,"(-1.375, 2.0)",6.836914e+00,1,233,0.000000,/mnt/data/plot rosenbrock trajectory_powell_ba...,/mnt/data/plot rosenbrock convergence_powell_b...
5,rosenbrock,"(-1.5, 2.0)",CCD-Accel,backtracking,"(-1.375, 2.0)",6.836914e+00,1,236,0.004111,/mnt/data/plot rosenbrock trajectory_ccd_backt...,/mnt/data/plot rosenbrock convergence_ccd_back...
6,ackley,"(4.0, 1.0)",Powell,backtracking,"(4.0, 1.0000000000000022)",8.836639e+00,0,116,0.002052,/mnt/data/plot ackley trajectory_powell_backtr...,/mnt/data/plot ackley convergence_powell_backt...
7,ackley,"(4.0, 1.0)",CCD-Accel,backtracking,"(4.0, 1.0000000000000022)",8.836639e+00,0,116,0.007696,/mnt/data/plot ackley trajectory_ccd_backtrack...,/mnt/data/plot ackley convergence_ccd_backtrac...


In [9]:
# Cell 9: Save short discussion & zip key outputs
discussion_lines = []
for func in ['rosenbrock', 'ackley']:
    for ls in ['golden', 'backtracking']:
        sub = df_cmp[(df_cmp['function']==func) & (df_cmp['line_search']==ls)]
        if len(sub)==2:
            s_pow = sub[sub['method']=='Powell'].iloc[0]
            s_ccd = sub[sub['method']=='CCD-Accel'].iloc[0]
            winner_val = 'Powell' if s_pow['f(x found)'] < s_ccd['f(x found)'] else 'CCD-Accel'
            faster = 'Powell' if s_pow['time (s)'] < s_ccd['time (s)'] else 'CCD-Accel'
            fewer_eval = 'Powell' if s_pow['f evals'] < s_ccd['f evals'] else 'CCD-Accel'
            discussion_lines.append(f'[{func}, {ls}] Lower objective: {winner_val}; fewer evals: {fewer_eval}; faster: {faster}.')

DISC_PATH = '/mnt/data/powell_vs_ccd_summary.txt'
with open(DISC_PATH, 'w', encoding='utf-8') as f:
    f.write('\n'.join(discussion_lines))

PY_PATH = '/mnt/data/powell_and_linesearch.py'
# Also export minimal python implementation file for grading reference (same code structure as notebook cells)
with open(PY_PATH, 'w', encoding='utf-8') as fpy:
    fpy.write("""
# Minimal export: Powell, CCD-Accel, and line-searches (derivative-free)
import numpy as np, math, time
from dataclasses import dataclass, field
from typing import Callable, List

def norm(x): import numpy as _np; return float(_np.linalg.norm(x))

@dataclass
class FWrap:
    f: Callable
    name: str
    evals: int = 0
    history_x: List = field(default_factory=list)
    history_f: List = field(default_factory=list)
    def __call__(self, x):
        fx = float(self.f(x))
        self.evals += 1
        self.history_x.append(np.array(x, dtype=float))
        self.history_f.append(fx); return fx
    def reset(self): self.evals=0; self.history_x.clear(); self.history_f.clear()

def line_search_golden(f, x, d, a=-50.0, b=50.0, tol=1e-6):
    golden_ratio = (math.sqrt(5)-1)/2
    def phi(al): return f(x + al*d)
    a1 = a + (1-golden_ratio)*(b-a); a2 = a + golden_ratio*(b-a)
    f1 = phi(a1); f2 = phi(a2)
    while abs(b-a) > tol:
        if f1 < f2: b,a2,f2,a1 = a2,a1,f1, a + (1-golden_ratio)*(b-a); f1 = phi(a1)
        else:       a,a1,f1,a2 = a1,a2,f2, a + golden_ratio*(b-a);    f2 = phi(a2)
    al = 0.5*(a+b); x_new = x + al*d; return x_new, f(x_new), al

def line_search_backtracking(f, x, d, alpha0=1.0, rho=0.5, c=1e-4, max_backtracks=60):
    fx = f(x); import numpy as _np
    eps = 1e-6 / max(1.0, norm(d)); slope = (f(x + eps*d) - fx)/eps
    al = alpha0
    for _ in range(max_backtracks):
        xn = x + al*d
        if f(xn) <= fx + c*al*slope: return xn, f(xn), al
        al *= rho
    xn = x + al*d; return xn, f(xn), al

@dataclass
class OptResult:
    method: str; x0: np.ndarray; x: np.ndarray; f: float; nit_outer: int; fevals: int
    time_sec: float; history_x: List; history_f: List

def powell(fw: FWrap, x0, tol=1e-6, max_outer=200, linesearch='golden'):
    n = len(x0); U = [np.eye(n)[i].copy() for i in range(n)]
    ls = line_search_golden if linesearch=='golden' else line_search_backtracking
    x = np.array(x0, float); t0=time.time(); hx=[x.copy()]; hf=[fw(x)]; k=0
    while k<max_outer:
        xp = x.copy()
        for i in range(n):
            x, fx, _ = ls(fw, x, U[i]); hx.append(x.copy()); hf.append(fx)
        for i in range(n-1): U[i]=U[i+1].copy()
        U[-1] = x - xp; x, fx, _ = ls(fw, x, U[-1]); hx.append(x.copy()); hf.append(fx)
        if norm(x-xp) <= tol: break
        k += 1
    T=time.time()-t0; return OptResult(f'Powell({linesearch})', np.array(x0), x, fw(x), k, fw.evals, T, hx, hf)

def ccd_accel(fw: FWrap, x0, tol=1e-6, max_outer=200, linesearch='golden'):
    ls = line_search_golden if linesearch=='golden' else line_search_backtracking
    x = np.array(x0, float); t0=time.time(); hx=[x.copy()]; hf=[fw(x)]; k=0; n=len(x0)
    while k<max_outer:
        xp = x.copy()
        for i in range(n):
            e = np.zeros(n); e[i]=1.0
            x, fx, _ = ls(fw, x, e); hx.append(x.copy()); hf.append(fx)
        d = x - xp; x, fx, _ = ls(fw, x, d); hx.append(x.copy()); hf.append(fx)
        if norm(x-xp) <= tol: break
        k += 1
    T=time.time()-t0; return OptResult(f'CCD-Accel({linesearch})', np.array(x0), x, fw(x), k, fw.evals, T, hx, hf)
""")

# Zip plots (if generated in this session) + code + discussion
ZIP_PATH = '/mnt/data/powell_outputs.zip'
with zipfile.ZipFile(ZIP_PATH, 'w', zipfile.ZIP_DEFLATED) as z:
    for p in os.listdir('/mnt/data'):
        if p.endswith('.png'):
            z.write(os.path.join('/mnt/data', p), arcname=p)
    z.write(DISC_PATH, arcname=os.path.basename(DISC_PATH))
    z.write(PY_PATH, arcname=os.path.basename(PY_PATH))

ZIP_PATH, DISC_PATH, PY_PATH

('/mnt/data/powell_outputs.zip',
 '/mnt/data/powell_vs_ccd_summary.txt',
 '/mnt/data/powell_and_linesearch.py')